In [2]:
import re

In [3]:
import pandas as pd

In [4]:
import requests
from bs4 import BeautifulSoup

In [8]:
from playwright.async_api import async_playwright

In [5]:
import csv
from datetime import datetime

Scraping the data from Japan

In [14]:
response_jp = requests.get("https://monitoring.tmiph.metro.tokyo.lg.jp/en/")
doc_jp = BeautifulSoup(response_jp.text)

In [15]:
shinjuku = doc_jp.find(class_="card tcard", id="shinjukuL").get_text()
shinjuku_data = re.findall(r'\d+\.\d+', shinjuku)[0]

Scraping the data from Taiwan

In [16]:
response_tw = requests.get("https://www.tldep.gov.taipei/EIACEP_EN/")
doc_tw = BeautifulSoup(response_tw.text)

In [17]:
rows = doc_tw.find_all('tr')
for row in rows:
    if 'DAZHI' in row.get_text():
        radiation_cell = row.find('td', title="Radiation")
        if radiation_cell:
            dazhi_data = radiation_cell.get_text()
            break

Scraping the data from Singapore

In [18]:
response_sg = requests.get("https://www.nea.gov.sg/our-services/radiation-safety/environmental-radiation-monitoring")
doc_sg = BeautifulSoup(response_sg.text)

In [19]:
rows = doc_sg.find_all('tr')
central_data = None
for row in rows:
    if 'Central' in row.get_text():
        central_data = row.find_all('td')[1].get_text().strip().split(' ')[0]

Scraping the data from Hong Kong

In [20]:
response_hk = requests.get('https://www.hko.gov.hk/radiation/monitoring/data/rmn_hourly_mean_used.txt?_=1701369406607')
soup = BeautifulSoup(response_hk.text, 'html.parser')

In [21]:
text = soup.get_text()
lines = text.split('\n')
kp_data = None
for line in lines:
    if line.startswith(" KP"):
        kp_data = line.split()[-1]

In [22]:
response_cn = requests.get("http://ditu.92cha.com/kqxsjll.php")
soup_cn = BeautifulSoup(response_cn.text, "html.parser")

In [23]:
table = soup_cn.find('table', class_='table')
for row in table.find_all('tr'):
    columns = row.find_all('td')
    if columns:
        province, station, measurement_time, value = [col.text.strip() for col in columns]
        if station == "宁波宝善路站":
            nb_data = float(value) / 1000
        if station == "奥林匹克森林公园站":
            bj_data = float(value) / 1000
        if station == "浦东新区临港站":
            sh_data = float(value) / 1000
        if station == "广州大道站":
            can_data = float(value) / 1000

In [45]:
headers = ["Time", "Tokyo", "Hong Kong", "Taipei", "Singapore", "Beijing", "Shanghai", "Guangzhou", "Thomas's Hometown"]

new_data_row = [datetime.now().strftime("%Y-%m-%d %H:%M:%S"), shinjuku_data, kp_data, dazhi_data, 
                central_data, bj_data, sh_data, can_data, nb_data]

file_path = 'radiation_log.csv'


with open('radiation_log.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    
    writer.writerow(new_data_row)

In [8]:
file_path = 'radiation_log_week.csv'

# Append the new data row to the CSV file
with open(file_path, 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(new_data_row)

# Read the data from the CSV file
with open(file_path, 'r', newline='') as file:
    reader = csv.reader(file)
    headers = next(reader)  # Read the header row
    all_rows = list(reader)

# Keep only the last 7 rows
rows_to_keep = all_rows[-7:]

# Write the reduced data back to the CSV file
with open(file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(headers)  # Write the headers
    writer.writerows(rows_to_keep)  # Write the last 7 days' data

NameError: name 'new_data_row' is not defined

In [25]:
latest_radiation_data = {
    "Tokyo": shinjuku_data, 
    "Hong Kong": kp_data, 
    "Taipei": dazhi_data, 
    "Singapore": central_data, 
    "Beijing": bj_data, 
    "Shanghai": sh_data, 
    "Guangzhou": can_data, 
    "Ningbo": nb_data
}

with open('radiation_log_map.csv', 'r', newline='') as file:
    reader = csv.DictReader(file)
    fieldnames = reader.fieldnames
    current_data = list(reader)

for row in current_data:
    city = row["City"]
    if city in latest_radiation_data:
        row["Latest Data"] = latest_radiation_data[city]
    row["Time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

with open('radiation_log_map.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(current_data)